In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
import os
from pathlib import Path

In [2]:
GANANCIA_ACIERTO = 780000
COSTO_ESTIMULO = -20000

In [3]:
# --- Paths Base (Ajustados a tu estructura) ---
BUCKET_PATH = "/home/sanmartinofacundo/buckets/b1"
EXP_ROOT = os.path.join(BUCKET_PATH, "exp")

# --- Configuración del Experimento ---
PARAM = {
    "experimento": "z_us50_python",
    "semilla_primigenia": 974411,
    
    # TU NUEVO INPUT PATH (Parquet)
    "input_path": "/home/sanmartinofacundo/buckets/b1/features/fe_v10/competencia_03_fe_v10.parquet",
    
    "output_path": EXP_ROOT,
    
    # Meses que definiste anteriormente
    "train_meses": [
        201901, 201902, 201903, 201904, 201905, 201906,
        201907, 201908, 201909, 201910, 201911, 201912,
        202001, 202002, 202003, 202004, 202005, 202006,
        202007, 202008, 202009, 202010, 202011, 202012,
        202101, 202102, 202103, 202104, 202105
    ],
    "future_mes": [202107], # Ajustado a tu validación externa o test
    
    "undersampling": 0.2,
    "qcanaritos": 5,
}

# Crear directorios
experimento_folder = os.path.join(PARAM["output_path"], PARAM["experimento"])
os.makedirs(experimento_folder, exist_ok=True)
os.makedirs(os.path.join(experimento_folder, "kaggle"), exist_ok=True)

print(f"Iniciando: {PARAM['experimento']}")
print(f"Input: {PARAM['input_path']}")

Iniciando: z_us50_python
Input: /home/sanmartinofacundo/buckets/b1/features/fe_v10/competencia_03_fe_v10.parquet


In [4]:
# --- Carga de Datos ---
print("Cargando dataset (formato Parquet)...")

# Lectura directa (pandas detecta pyarrow o fastparquet automáticamente)
df = pd.read_parquet(PARAM["input_path"])

print(f"Dimensiones cargadas: {df.shape}")

Cargando dataset (formato Parquet)...
Dimensiones cargadas: (4883051, 894)


In [5]:
# --- 4. Producción y Undersampling ---

# Filtrar dataset para entrenamiento final
df_train = df[df['foto_mes'].isin(PARAM['train_meses'])].copy()

# Undersampling
np.random.seed(PARAM['semilla_primigenia'])
df_train['azar'] = np.random.uniform(0, 1, size=len(df_train))

# Condición: Mantener todos los BAJA+1 y BAJA+2, y el 50% de CONTINUA
cond_train = (df_train['azar'] <= PARAM['undersampling']) | \
             (df_train['clase_ternaria'].isin(["BAJA+1", "BAJA+2"]))

df_train = df_train[cond_train].copy()
df_train.drop(columns=['azar'], inplace=True)

# Target Engineering (Binaria)
# BAJA+1 y BAJA+2 -> 1, CONTINUA -> 0
df_train['clase01'] = np.where(df_train['clase_ternaria'].isin(["BAJA+1", "BAJA+2"]), 1, 0)

print(f"Registros de entrenamiento final: {len(df_train)}")
print(df_train['clase01'].value_counts())

Registros de entrenamiento final: 873661
clase01
0    836598
1     37063
Name: count, dtype: int64


In [6]:
# --- 5. zLightGBM: Canaritos ---
# IMPORTANTE: En zLightGBM los canaritos deben ser las PRIMERAS columnas.

print(f"Agregando {PARAM['qcanaritos']} canaritos...")
canaritos_cols = []
for i in range(1, PARAM['qcanaritos'] + 1):
    col_name = f"canarito_{i}"
    # Generamos vector aleatorio
    df_train[col_name] = np.random.uniform(0, 1, size=len(df_train))
    canaritos_cols.append(col_name)

# Reordenar columnas: Canaritos primero + Features originales
features_reales = [c for c in df_train.columns if c not in canaritos_cols + ['clase_ternaria', 'clase01', 'training']]
# Nota: 'training' no existe aquí porque filtramos el df, pero lo excluimos por seguridad
cols_finales_ordenadas = canaritos_cols + features_reales

# Crear Dataset de LightGBM
dtrain = lgb.Dataset(
    data=df_train[cols_finales_ordenadas],
    label=df_train['clase01'],
    free_raw_data=False
)

# Limpiar df_train para liberar RAM antes de entrenar
del df_train
gc.collect()

Agregando 5 canaritos...


40

In [7]:
# --- 6. Configuración y Entrenamiento ---

# Parámetros zLightGBM
params = {
    # --- Parámetros Estándar (Mapeo Correcto) ---
    'boosting_type': 'gbdt',          # En R: boosting
    'objective': 'binary',
    'metric': 'custom',
    'boost_from_average': True,
    'force_row_wise': True,
    'verbosity': -1,                  # Equivale a -100 en R (silencioso)
    'seed': PARAM['semilla_primigenia'],
    'max_bin': 31,
    'min_data_in_leaf': 20,
    'num_leaves': 999,
    'learning_rate': 1.0,
    'feature_fraction': 0.5,
    
    # --- Parámetros Faltantes en tu versión anterior ---
    'feature_pre_filter': False,      # CRÍTICO: Para igualar a R
    'first_metric_only': False,       # Para igualar a R
    
    # --- Hiperparámetros específicos de zLightGBM ---
    'gradient_bound': 0.1,
    'canaritos': PARAM['qcanaritos']  # CORRECCIÓN: Usar 'canaritos' para machar C++
}

print("Entrenando zLightGBM...")

# num_boost_round va aquí como argumento, NO en el diccionario params
model = lgb.train(
    params,
    dtrain,
    num_boost_round=9999
)

# Guardar modelo
model.save_model(os.path.join(experimento_folder, f"zmodelo_{PARAM['undersampling']:.2f}.txt"))

Entrenando zLightGBM...


In [8]:
# --- 7. Scoring y Evaluación de Ganancia ---
print("Evaluando ganancias...")


# 1. Preparar datos futuros (Validación)
df_future = df[df['foto_mes'].isin(PARAM['future_mes'])].copy()

# Agregar canaritos (aleatorios)
for col in canaritos_cols:
    df_future[col] = np.random.uniform(0, 1, size=len(df_future))

# 2. Predecir
# Usamos cols_finales_ordenadas (o cols_ordenadas según como te quedó en el paso 5)
prediccion = model.predict(df_future[cols_finales_ordenadas])

# 3. Preparar la Realidad (y_true)
# Solo BAJA+2 genera ganancia. El resto (incluido BAJA+1) es costo si se envía.
y_true = np.where(df_future['clase_ternaria'] == "BAJA+2", 1, 0)

# 4. Calcular Curva de Ganancia Vectorizada
# a. Ordenar predicciones de mayor a menor
idx_sorted = np.argsort(prediccion)[::-1]
y_true_sorted = y_true[idx_sorted]

# b. Calcular ganancia individual de cada envío (vectorizado)
#    Si es BAJA+2 (1) -> Gana 780000 y Paga 20000 = +760000
#    Si es OTRO (0)   -> Solo Paga 20000          = -20000
valores_individuales = np.where(
    y_true_sorted == 1,
    GANANCIA_ACIERTO + COSTO_ESTIMULO,
    COSTO_ESTIMULO
)

# c. Ganancia acumulada
ganancia_acumulada = np.cumsum(valores_individuales)

# 5. Imprimir resultados para cada corte solicitado
CORTES_ENVIO = [9000, 9500, 10000, 10500, 11000, 11500, 12000, 12500, 13000, 13500, 14000, 14500, 15000]

print("\nResultados de Ganancia por Corte:")
print(f"{'Envios':>10} | {'Ganancia Total':>20}")
print("-" * 35)

for corte in CORTES_ENVIO:
    if corte <= len(ganancia_acumulada):
        # El índice es corte - 1 porque el array empieza en 0
        ganancia_corte = ganancia_acumulada[corte - 1]
        print(f"{corte:10d} | {ganancia_corte:20,.0f}")
    else:
        print(f"{corte:10d} | {'No hay suficientes datos':>20}")

# Opcional: Mostrar el mejor corte absoluto encontrado
mejor_indx = np.argmax(ganancia_acumulada)
mejor_ganancia = ganancia_acumulada[mejor_indx]
print("-" * 35)
print(f"MEJOR CORTE: {mejor_indx + 1} envíos | Ganancia: {mejor_ganancia:,.0f}")
print("Finalizado.")

Evaluando ganancias...

Resultados de Ganancia por Corte:
    Envios |       Ganancia Total
-----------------------------------
      9000 |          379,260,000
      9500 |          378,620,000
     10000 |          382,660,000
     10500 |          379,680,000
     11000 |          385,280,000
     11500 |          389,320,000
     12000 |          391,020,000
     12500 |          390,380,000
     13000 |          386,620,000
     13500 |          389,880,000
     14000 |          392,360,000
     14500 |          394,060,000
     15000 |          394,200,000
-----------------------------------
MEJOR CORTE: 14782 envíos | Ganancia: 397,000,000
Finalizado.


In [9]:
# # Guardar archivo para Kaggle
# archivo_salida = os.path.join(experimento_folder, "kaggle", f"KA{PARAM['experimento']}_{PARAM['corte_envios']}.csv")
# tb_prediccion[['numero_de_cliente', 'Predicted']].to_csv(archivo_salida, index=False)

# print(f"Archivo generado exitosamente: {archivo_salida}")
# print("¡Éxito en la competencia!")